In [13]:
import pandas as pd
import os
import gc

## Processing from Raw MBSTAR Datasets 

In [ ]:
dirnames = os.listdir('./MBStar/MBSTAR_Genome_Wide_pred_res')
cols = ['miRNA', 'mRNA', 'Binding sites with flanking regions', 'miRNA Sequence', 'Score']
savepath1 = './Processed MBStar/miRNA-wise'

In [ ]:
total_rows = 0
for each in dirnames:
    filenames = os.listdir(os.path.join('./MBStar/MBSTAR_Genome_Wide_pred_res',each))
    df = pd.read_csv(os.path.join('./MBStar/MBSTAR_Genome_Wide_pred_res',each,filenames[0]), 
                     sep = '\t', skiprows=1, usecols = cols)
    for file in filenames[1:]:
        df = df.append(pd.read_csv(os.path.join('./MBStar/MBSTAR_Genome_Wide_pred_res',each,file), 
                     sep = '\t', skiprows=1, usecols = cols))
    df = df[df.isnull().T.any()==False]
    df.to_csv(os.path.join(savepath1, '%s.txt'%(each)), sep = '\t')
    total_rows += df.shape[0]

In [ ]:
total_rows

In [ ]:
total_rows

In [ ]:
partition = int(total_rows/2)

In [ ]:
file

In [ ]:
df.isnull().sum()

In [ ]:
df[df.isnull().T.any()==True]

## Partitioning
Dividing into Datasets for Next Sentance Prediction and Masked Language Modelling

In [2]:
filenames = os.listdir('./Processed MBStar/miRNA-wise')
cols = ['miRNA', 'mRNA', 'Binding sites with flanking regions', 'miRNA Sequence', 'Score']
savepath2 = './Processed MBStar/partitioned/mlm'
savepath3 = './Processed MBStar/partitioned/nsp'
partition = 23732461 # 1/2 of the total rows

In [3]:
filenames

['output10.txt',
 'output8.txt',
 'output4.txt',
 'output1.txt',
 'output20.txt',
 'output3.txt',
 'output15.txt',
 'output18.txt',
 'output9.txt',
 'output16.txt',
 'output17.txt',
 'output2.txt',
 'output12.txt',
 'output6.txt',
 'output14.txt',
 'output11.txt',
 'output19.txt',
 'output7.txt',
 'output5.txt',
 'output13.txt']

### For Masked Language Modelling Adaptation
50% of the partition is used for Masked Language Modelling.

In [19]:
mlm_rows = 0
prev_rows = 0
c = 0
flag = False

for idx1, each in enumerate(filenames):
    if idx1 == 0 or flag:
        df = pd.read_csv(os.path.join('./Processed MBStar/miRNA-wise',each), 
                     sep = '\t', skiprows=0, index_col=0)
        prev_rows = df.shape[0]
        flag = False
    else:
        df = df.append(pd.read_csv(os.path.join('./Processed MBStar/miRNA-wise',each), 
                         sep = '\t', index_col=0))
#     print(df.head())
    
    mlm_rows += df.shape[0] if idx1 == 0 or (idx1%3==1 and idx1>1) or flag else df.shape[0] - prev_rows
    prev_rows = df.shape[0]
    
    print(idx1, df.shape[0], mlm_rows, each)
    
    if idx1%3==0 and idx1 != 0:
        miRNA_df = df[['miRNA', 'miRNA Sequence']]
        mRNA_df = df[['mRNA', 'Binding sites with flanking regions']]
        
        miRNA_df.to_csv(os.path.join(savepath2, 'mlm_mirna_data_chunk_{}.txt'.format(c)), sep='\t')
        mRNA_df.to_csv(os.path.join(savepath2, 'mlm_mrna_data_chunk_{}.txt'.format(c)), sep='\t')
        
        c+=1
        flag = True
        print("Saving chunk...",c)
        del [[df, miRNA_df, mRNA_df]]
        gc.collect()
    
    if mlm_rows >= partition:
        idx_nsp = idx1+1
        miRNA_df = df[['miRNA', 'miRNA Sequence']]
        mRNA_df = df[['mRNA', 'Binding sites with flanking regions']]
        
        miRNA_df.to_csv(os.path.join(savepath2, 'mlm_mirna_data_chunk_{}.txt'.format(c)), sep='\t')
        mRNA_df.to_csv(os.path.join(savepath2, 'mlm_mrna_data_chunk_{}.txt'.format(c)), sep='\t')
        break

print(mlm_rows, partition, mlm_rows, idx_nsp)
        

/home/abhik/anaconda3/envs/pytorch_env/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


0 1954317 1954317 output10.txt
1 4118035 4118035 output8.txt
2 6375433 6375433 output4.txt
3 9180167 9180167 output1.txt
Saving chunk... 1
4 3304595 12484762 output20.txt
5 5590041 14770208 output3.txt
6 7782324 16962491 output15.txt
Saving chunk... 2
7 2062929 19025420 output18.txt
8 3941953 20904444 output9.txt
9 6020687 22983178 output16.txt
Saving chunk... 3
10 2167140 25150318 output17.txt
25150318 23732461 25150318 11


In [21]:
del miRNA_df, mRNA_df
gc.collect()

4109

### For Next Sentance Prediction Adaptation
Remaining 50% of partition is used for NSP

In [25]:
nsp_rows = 0
c = 0
flag = False

for idx2, each in enumerate(filenames[idx_nsp+1:]):
    if idx2 == 0 or flag:
        df = pd.read_csv(os.path.join('./Processed MBStar/miRNA-wise',each), 
                     sep = '\t', index_col=0)
        prev_rows = df.shape[0]
    else:
        df = df.append(pd.read_csv(os.path.join('./Processed MBStar/miRNA-wise',each), 
                         sep = '\t', index_col=0))
#     print(df.head())
    
    
    nsp_rows += df.shape[0] if idx2 == 0 or (idx2%3==1 and idx2>1) or flag else df.shape[0] - prev_rows
    prev_rows = df.shape[0]
    print(idx2, df.shape[0], nsp_rows, each)
    
    if idx2%3==0 and idx2 != 0:
        df.to_csv(os.path.join(savepath3, 'nsp_data_chunk_{}.txt'.format(c)), sep='\t')
        print("Saving chunk...",c)
        c += 1
        flag = True
        del df
        gc.collect()

df.to_csv(os.path.join(savepath3, 'nsp_data_chunk_{}.txt'.format(c)), sep='\t')
print("Saving chunk...",c)
del df
gc.collect()        

print(nsp_rows, partition, idx2)

0 2222301 2222301 output12.txt
1 5083876 5083876 output6.txt
2 7390866 7390866 output14.txt
3 9401831 9401831 output11.txt
Saving chunk... 0
4 2133259 11535090 output19.txt
5 4021807 15556897 output7.txt
6 2201504 17758401 output5.txt
Saving chunk... 1
7 2315103 20073504 output13.txt
Saving chunk... 2
20073504 23732461 7
